In [7]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)

from weighted_accuracy_and_tools import decompose_y, reconstruct_y, weighted_accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd

In [8]:
X_path= os.path.join("..", "..", "data","enriched_input", "X_train.csv")
X = pd.read_csv(X_path, delimiter=',')

y_path= os.path.join("..", "..", "data","enriched_input", "y_train.csv")
y = pd.read_csv(y_path, delimiter=',')

In [9]:
X.set_index("DELIVERY_START", inplace=True)
y.set_index("DELIVERY_START", inplace=True)
X.index = pd.to_datetime(X.index, utc=True)
y.index = pd.to_datetime(y.index, utc=True)

In [10]:
X

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,cold_rate,av_consuption_hour,hour,dayofweek,quarter,month,year,dayofyear
DELIVERY_START,,,,,,,,,,,,,,,,
2022-01-01 01:00:00+00:00,-0.405599,1.143744,0.049156,1.145111,-0.441788,-0.709854,-0.349637,-0.580887,-1.112973,-0.775054,-1.519140,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 02:00:00+00:00,-0.682678,1.143744,0.049156,1.145111,-0.409506,-0.709854,-0.482002,-0.580887,-1.112973,-1.010691,-1.374581,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 03:00:00+00:00,-0.810713,1.143744,0.049156,1.145111,-0.366164,-0.709854,-0.614471,-0.580887,-1.112973,-1.565537,-1.230023,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 04:00:00+00:00,-0.846578,1.143744,0.049156,1.145111,-0.318637,-0.709854,-0.676317,-0.580887,-1.112973,-1.991068,-1.085464,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 05:00:00+00:00,-0.798790,1.143744,0.049156,1.145111,-0.249289,-0.709854,-0.715292,-0.580887,-1.112973,-2.026920,-0.940906,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29 17:00:00+00:00,-0.275482,1.143744,1.010389,0.384129,0.908396,-0.469079,0.924332,-0.399128,-0.492362,-0.085946,0.793798,-0.489343,-1.045267,-0.734099,2.020864,-0.618926
2023-03-29 18:00:00+00:00,-0.293084,1.143744,1.010389,0.384129,1.143341,-0.669539,0.230275,-0.522000,-0.492362,0.271836,0.938356,-0.489343,-1.045267,-0.734099,2.020864,-0.618926
2023-03-29 19:00:00+00:00,-0.522752,1.143744,1.010389,0.384129,1.375595,-0.709854,0.011252,-0.580887,-0.492362,0.969350,1.082915,-0.489343,-1.045267,-0.734099,2.020864,-0.618926


In [11]:
y_direction, y_magnitude = decompose_y(y['spot_id_delta'])

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np

# Artificially create feature names for demonstration
feature_names = [f"feature_{i}" for i in range(X.shape[1])]

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_magnitude, test_size=0.2, random_state=42)

# Define a pipeline that includes Sequential Feature Selector and Linear Regression
sfs_linear_pipeline = Pipeline([
    ('sfs', SequentialFeatureSelector(LinearRegression(), n_features_to_select=10, direction='forward')),
    ('reg', LinearRegression())
])

# Train the pipeline on the training data
sfs_linear_pipeline.fit(X_train, y_train)

# Extract the support of the selected features
selected_features = sfs_linear_pipeline.named_steps['sfs'].get_support()

# Extract the names of the selected features
selected_feature_names = [feature_names[i] for i in range(len(selected_features)) if selected_features[i]]

# Create the voting regressor with all models, including our pipeline
voting_reg = VotingRegressor(
    estimators=[
        ('sfs_linear_pipeline', sfs_linear_pipeline),
        ('decision_tree_reg', DecisionTreeRegressor(random_state=42)),
        ('random_forest_reg', RandomForestRegressor(random_state=42)),
        ('gradient_boosting_reg', GradientBoostingRegressor(random_state=42)),
        ('knn_reg', KNeighborsRegressor())
    ]
)

# Train the voting regressor on the original dataset
voting_reg.fit(X_train, y_train)

# Make predictions and evaluate the voting regressor
y_pred_voting = voting_reg.predict(X_test)
rmse_voting = np.sqrt(mean_squared_error(y_test, y_pred_voting))

rmse_voting, selected_feature_names


(17.249830788846133,
 ['feature_0',
  'feature_1',
  'feature_2',
  'feature_4',
  'feature_6',
  'feature_8',
  'feature_9',
  'feature_10',
  'feature_13',
  'feature_15'])

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Assuming X is a DataFrame and y_direction is your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y_direction, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Select features based on importance
selector = SelectFromModel(rf, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Get the mask of selected features
selected_features_mask = selector.get_support()

# Get the names of the selected features
selected_feature_names = X_train.columns[selected_features_mask].tolist()

print("Selected features:", selected_feature_names)

# Re-instantiate the individual classifiers
knn = KNeighborsClassifier(n_neighbors=5)
dtree = DecisionTreeClassifier(random_state=42)
logreg = LogisticRegression(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
gradient_boosting = GradientBoostingClassifier(random_state=42)

# Create the extended voting classifier with all models, using selected features
extended_voting_clf = VotingClassifier(
    estimators=[
        ('knn', knn),
        ('dtree', dtree),
        ('logreg', logreg),
        ('random_forest', random_forest),
        ('gradient_boosting', gradient_boosting)
    ],
    voting='hard'
)

# Train the extended voting classifier on scaled data
extended_voting_clf.fit(X_train[selected_feature_names], y_train)

# Make predictions and evaluate the extended model
y_pred = extended_voting_clf.predict(X_test[selected_feature_names])

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the voting classifier on selected features:", accuracy)


c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Selected features: ['load_forecast', 'nucelear_power_available', 'wind_power_forecasts_average', 'wind_power_forecasts_std', 'hour', 'dayofweek', 'dayofyear']
Accuracy of the voting classifier on selected features: 0.7868929750117869
